In [12]:
import cv2
import numpy as np
import time

# Algoritmo A*

Miembros:
- Rafael Andrade
- Joao Basuri
- Dante Moreno

Definimos las dimensiones del mapa

In [2]:
largo=5
ancho=5

* ***mapa:*** Arreglo que representa al mapa.
* ***adyacentes(n):*** Si ***mapa*** es un arreglo que empieza en 0; devuelve el conjunto de otras ubicaciones (también como números) adyacentes a esta ubicación.

* $frame$: Representación gráfica del mapa, de ($ancho * 50) \times (largo * 50$)

In [3]:
mapa = ['O','O','O','O','O',
        'O','O','X','O','O',
        'I','O','X','O','F',
        'O','O','X','O','O',
        'O','O','O','O','O']
posini = -1
posfin = -1
intran = set([])
for i in range(len(mapa)):
  if mapa[i]=='X':
    intran.add(i)
  if mapa[i]=='I':
    posini = i
  if mapa[i]=='F':
    posfin = i

In [4]:
frame = np.zeros((largo*50,ancho*50,3),dtype=np.uint8)

In [5]:
#frame = np.array(frame)
print(type(frame))

<class 'numpy.ndarray'>


In [5]:
frame.shape

(250, 250, 3)

In [6]:
def adyacentes(num):
  lista = set([])
  if num%ancho and num>ancho and mapa[num-ancho-1]!='X':
    lista.add(num-ancho-1)
  if (num+1)%ancho and num>=ancho and mapa[num-ancho+1]!='X':
    lista.add(num-ancho+1)
  if num>=ancho and mapa[num-ancho]!='X':
    lista.add(num-ancho)
  if num%ancho and mapa[num-1]!='X':
    lista.add(num-1)
  if (num+1)%ancho and mapa[num+1]!='X':
    lista.add(num+1)
  if num<ancho*largo-ancho and mapa[num+ancho]!='X':
    lista.add(num+ancho)
  if num%ancho and num<ancho*largo-ancho and mapa[num+ancho-1]!='X':
    lista.add(num+ancho-1)
  if (num+1)%ancho and num<ancho*largo-ancho and mapa[num+ancho+1]!='X':
    lista.add(num+ancho+1)
  return lista

adyacentes(11)

{5, 6, 10, 15, 16}

* ***getcoor(num)***: Dado un número ***num***, retorna las posiciones *(x,y)* correspondientes a ***mapa***.
* ***dist(num)***: Dado un número ***num***, retorna la distancia Manhattan hacia la posición de destino ***posfin***.
* ***iscorner(ini,fin)***: Dadas dos posiciones ***ini*** y ***fin***, se retorna si una es esquina de otra.

In [7]:
def getcoor(num):
  return(int(num%ancho),int(num/ancho))

def dist(num):
  x,y = getcoor(num)
  xf,yf = getcoor(posfin)
  return 10*(abs(x-xf)+abs(y-yf))

def iscorner(ini,fin):
  if ini-ancho-1==fin or ini-ancho+1==fin or ini+ancho-1==fin or ini+ancho+1==fin:
    return True
  else:
    return False

* $pintar_num_img(num,bgr)$: Función que pinta en $frame$ una coordenada específica (según la representación del arreglo lineal $mapa$)

In [8]:
def pintar_num_img(num, bgr_arr): #Con la misma lógica que getcoor, pero adaptado a la imagen
    (x,y) = getcoor(num)
    X = x * 50
    Y = y * 50
    frame[Y:Y+50,X:X+50] = np.array(bgr_arr,dtype=np.uint8)

In [9]:
for muro in intran: 
    pintar_num_img(muro,[20,20,255])
pintar_num_img(posini,[20,255,20])
pintar_num_img(posfin,[255,20,20])

### Implementación del algoritmo A*

In [16]:
openList = []
closeList = []
openList.append(posini)
fslist = [0 for i in range(largo*ancho)]
padres = [-1 for i in range(largo*ancho)]
endNode = None
FinalCost = 0
fslist[posini] = dist(posini)
while True:
    time.sleep(0.6)
    if openList == []:
        print("No se puede llegar al destino")
        exit(1)
    n = min(openList, key=lambda x: fslist[x])
    # pintar Mapa
    for i in range(ancho*largo):
        if not {i}.issubset(intran):
            pintar_num_img(i,[10,10,10])
    pintar_num_img(posini,[20,180,20])
    pintar_num_img(posfin,[255,20,20])
    # --- end pintar Mapa
    openList.remove(n)
    #for elem in openList:
    #    pintar_num_img(elem,[10,100,10]) #Pinta la lista abierta
    #for elem in closeList:
    #    pintar_num_img(elem,[10,50,50]) #Pinta la lista cerrada
    closeList.append(n)
    pintar_num_img(n,[0,255,0]) #Pinta el nodo elegido
    if n == posfin:
        #pintar_num_img(n,[255,255,255])
        endNode = n
        FinalCost = fslist[n]
        break
    ady = adyacentes(n)
    gs = fslist[n] - dist(n)
    for i in ady:
        s = 10
        if iscorner(n,i):
            s = 14
        if i in openList:
            if fslist[i] > gs + s + dist(i):
                fslist[i] = gs + s + dist(i)
                padres[i] = n
        else:
            if i in closeList:
                if fslist[i] > gs + s + dist(i):
                    fslist[i] = gs + s + dist(i)
                    padres[i] = n
                    openList.append(i)
                    closeList.remove(i)
            else:
                fslist[i] = gs + s + dist(i)
                padres[i] = n
                openList.append(i)
        
    cv2.imshow('A Star Laberinto', frame) #...Imprime el frame
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.imshow('A Star Laberinto', frame) #...Imprime el frame
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
        
n = endNode
sol = []
while True:
    sol.append(n)
    if padres[n] == -1:
        break
    n = padres[n]
sol.reverse()
print("Ruta: ",list(sol))
print("Costo final: ", FinalCost)

Ruta:  [10, 6, 2, 8, 14]
Costo final:  56
